<a href="https://colab.research.google.com/github/tinkvu/TravelRecommenderUsingLLM/blob/main/Notebooks/google_t5_flan_base_without_lora_config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
ERROR: pip's depe

In [2]:
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [3]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
import pandas as pd
from datasets import Dataset
# Assuming 'series.csv' is in the current directory
df = pd.read_csv('series.csv')

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)


In [6]:
type(dataset)

datasets.arrow_dataset.Dataset

In [16]:
def summarize_review(example):
    input_text = "there is a place name and reviews written by many people. Summarize these reviews and create a description for the place: " + example['context']
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Generate summary
    outputs = model.generate(input_ids)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"*********************\nContext: {context}\nOutput: {summary}")
    return {'summary': summary}

# Apply the function to each example in the dataset
dataset = dataset.map(summarize_review)

# Print the resulting dataset with the new summary column
print(dataset)

Map:   0%|          | 0/217 [00:00<?, ? examples/s]

*********************
Context: Place Name: 14 Henrietta Street, Reviews: The museum is very good. I highly recommend it to know more about georgian houses and Dublins life. The experience was overall really good. However, the tour was with a group of 20 people, and the rooms are not big enough to appreciate all with big groups. The museum should consider this. The guide was ok, good information but with no group management at all. He started to talk before the group was inside the room. Ive lost half of the tour because of this. This museum tells an important story about perseverance, community building, social class, and the spirit struggle for social justice in Ireland with lessons all societies. 14 Henrietta invites visitors to look inside this Irish history with empathy and build capacity to confront disparities in any way one can. Excellent tour narrative and engaging use of technology. A must visit for locals and tourists. A real look into tenement life in Dublin and the history 

KeyboardInterrupt: 

In [10]:
dataset['context'][0]

'Place Name: 14 Henrietta Street, Reviews: The museum is very good. I highly recommend it to know more about georgian houses and Dublins life. The experience was overall really good. However, the tour was with a group of 20 people, and the rooms are not big enough to appreciate all with big groups. The museum should consider this. The guide was ok, good information but with no group management at all. He started to talk before the group was inside the room. Ive lost half of the tour because of this. This museum tells an important story about perseverance, community building, social class, and the spirit struggle for social justice in Ireland with lessons all societies. 14 Henrietta invites visitors to look inside this Irish history with empathy and build capacity to confront disparities in any way one can. Excellent tour narrative and engaging use of technology. A must visit for locals and tourists. A real look into tenement life in Dublin and the history on how it started. Lloyd was o

In [ ]:
# We prefix our tasks with "answer the question"
prefix="As a travel guide, answer to the user queries about Dublin: "
#prefix = "Please answer this question: "

# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["Questions"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)

   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["Answers"],
                      max_length=512,
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [ ]:
# Tokenize datasets
tokenized_train_dataset = train_loaded_data.map(preprocess_function, batched=True, remove_columns=["Questions", "Answers"]
)

tokenized_eval_dataset = val_loaded_data.map(preprocess_function, batched=True, remove_columns=["Questions", "Answers"]
)

Map:   0%|          | 0/1019 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1019
})

In [ ]:
len(tokenized_train_dataset)

1019

In [ ]:
tokenized_eval_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 114
})

In [ ]:
from peft import LoraConfig, get_peft_model

In [ ]:
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="SEQ_2_SEQ_LM",
)

In [ ]:
def formatting_func(example):
    text = f"Question: \n{example['Question'][0]}\n\nAnswer: \n{example['Answer'][0]}"
    return [text]

In [ ]:
import math

In [ ]:
# Global Parameters
L_RATE = 0.001
BATCH_SIZE = 4
gradient_accumulation_steps = 2
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 5
STEPS = len(tokenized_train_dataset) / (BATCH_SIZE * gradient_accumulation_steps)
number_of_epochs = 5
MAX_STEPS = int(number_of_epochs * STEPS)
# MAX_STEPS = math.floor(MAX_STEPS)
WARM_UP_STEPS = int(0.01 * MAX_STEPS)
# WARM_UP_STEPS = math.floor(WARM_UP_STEPS)
LOGGING_STEPS = 50
SAVE_STEPS = 200


# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="steps",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   predict_with_generate=True,
   push_to_hub=False,
   gradient_accumulation_steps=gradient_accumulation_steps,
   max_steps=MAX_STEPS,
   warmup_steps=WARM_UP_STEPS,
   logging_steps=LOGGING_STEPS,
   eval_steps = LOGGING_STEPS,
   generation_max_length=512,
   save_steps=SAVE_STEPS  # Adding checkpointing

)

In [ ]:
MAX_STEPS

636

In [ ]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_eval_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
50,2.616000,2.259331
100,2.404500,2.144463
150,2.149500,2.058720
200,1.888100,2.009343
250,1.854300,1.963397
300,1.569800,1.992152
350,1.552000,1.949653
400,1.399400,2.020042
450,1.264900,2.008219
500,1.265700,1.974201


TrainOutput(global_step=636, training_loss=1.6467940672388617, metrics={'train_runtime': 521.8386, 'train_samples_per_second': 9.75, 'train_steps_per_second': 1.219, 'total_flos': 190353182063616.0, 'train_loss': 1.6467940672388617, 'epoch': 4.99})

In [ ]:
import pickle

# Load the pickle file
with open('validation_dataset.pkl', 'rb') as file:
    data = pickle.load(file)

# Extract questions from the data
questions = []
for item in data:
    if 'Questions' in item:
        questions.append(item['Questions'])

# Write questions to a text file
with open('Questions.txt', 'w', encoding='utf-8') as file:
    for question in questions:
        file.write(question + '\n')

print("Questions have been extracted and saved to Questions.txt")

Questions have been extracted and saved to Questions.txt


In [ ]:
import os
import time

def generate_response(prompt, max_length=520):
    """Generate response from the model for a given prompt."""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(inputs["input_ids"], max_length=max_length, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def real_time_testing(*args):
    """
    Process either a single question or a text file of questions. For a single question, print the response.
    For a text file, save the questions and generated answers to a text file.

    Args:
        *args: Single question as a string or text file path as a string.
    """
    questions = []
    is_single_question = False

    # Check if the input is a single question or a file path
    if len(args) == 1 and isinstance(args[0], str):
        input_arg = args[0]
        if os.path.isfile(input_arg):  # Check if the input is a file path
            txt_filename = input_arg
            # Read questions from text file
            try:
                with open(txt_filename, 'r', encoding='utf-8') as txt_file:
                    questions = [line.strip() for line in txt_file.readlines() if line.strip()]
            except IOError as e:
                print(f"Error reading text file {txt_filename}: {e}")
                return
        else:  # Assume the input is a single question
            questions.append(input_arg)
            is_single_question = True
    else:
        print("Invalid input. Please provide a single question or a text file path.")
        return

    # Initialize variables to hold instruction-output pairs
    instruction_output_pairs = []

    start = time.time()

    # Generate outputs for each question and collect pairs
    for idx, question in enumerate(questions):
        generated_text = generate_response(question)
        instruction_output_pairs.append((question, generated_text))
        if is_single_question:
            print(f"*******************\n{idx + 1}. Prompt: {question} \nAnswer: {generated_text}\n")
        else:
            print(f"*******************\n{idx + 1}. Prompt: {question} \nAnswer: {generated_text}\n")

    end = time.time()
    print(f"Time taken: {end - start} seconds")

    if not is_single_question:
        # Define the output text file path
        output_txt_file = 'generated_outputs.txt'

        # Write instruction-output pairs to text file
        try:
            with open(output_txt_file, 'w', encoding='utf-8') as txt_file:
                for pair in instruction_output_pairs:
                    txt_file.write(f"Question: {pair[0]}\nAnswer: {pair[1]}\n\n")
            print(f"Text file with generated outputs saved: {output_txt_file}")
        except IOError as e:
            print(f"Error writing to text file {output_txt_file}: {e}")

# Example usage
# real_time_testing("What is the EU AI act?")
# real_time_testing("questions.txt")

In [ ]:
real_time_testing("What is the EU AI act?")

*******************
1. Prompt: What is the EU AI act? 
Answer: The EU AI Act is a comprehensive legal framework aimed at ensuring the safe and ethical use of artificial intelligence (AI) technologies in the European Union (EU). It establishes provisions for the development, deployment, and use of AI in the EU market, promoting innovation while safeguarding fundamental rights and values.

Time taken: 5.504937410354614 seconds


In [ ]:
real_time_testing("Questions.txt")

*******************
1. Prompt: What are the obligations for deployers of high-risk AI systems in the enterprise? 
Answer: Deployers of high-risk AI systems in the enterprise bear several key obligations under the AI Act, including conducting thorough risk assessments, implementing effective risk mitigation measures, and ensuring continuous monitoring of system performance.

*******************
2. Prompt: What is the role of data protection officers (DPOs) in ensuring compliance with high-risk AI regulations? 
Answer: Data Protection Officers (DPOs) play a pivotal role in ensuring compliance with high-risk AI regulations under the EU AI Act. They are responsible for overseeing compliance with the requirements of the high-risk AI Act (Articles 10 and 17). They ensure that high-risk AI systems are designed and developed in a way that respects fundamental rights, values, and principles. They are responsible for ensuring that AI systems are developed and used in a manner that does not pose 